### General rules:
 * For all figures that you generate, remember to add meaningful labels to the axes (including units), and provide a legend and colorbar, if applicable.
 * Do not hard code constants, like number of samples, number of channels, etc in your program. These values should always be determined from the given data. This way, you can easily use the code to analyse other data sets.
 * Do not use high-level functions from toolboxes like scikit-learn.
 * Before submitting, check your code by executing: Kernel -> Restart & run all.
 * Replace *Template* by your *FirstnameLastname* in the filename, or by *Lastname1Lastname2* if you work in pairs.

# BCI-IL WS 2018/2019 - Exercise Sheet #03

#### Name: 

In [1]:
% matplotlib inline
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

import bci_minitoolbox as bci

## Exercise 1: Nearest Centroid Classifier (NCC)  (2 point)
Implement the calculation of the nearest centroid classifier (NCC) as a Python function `train_NCC`.  The function should take two arguments, the first being the data matrix $\bf{X}$ where each column is a data point ($\bf{x_k}$), and the second being class labels of the data points. Two output arguments should return the weight vector **`w`** and bias `b`.

In [2]:
def train_NCC(X, y):
    '''
    Synopsis:
        w, b= train_NCC(X, y)
    Arguments:
        X: data matrix (features X samples)
        y: labels with values 0 and 1 (1 x samples)
    Output:
        w: NCC weight vector
        b: bias term
    '''
    u1 = np.mean(X[:, y == 0])
    u2 = np.mean(X[:, y == 1])
    w = u2 - u1
    b = w.T*((u1 + u2)/2)
    return w,b

## Exercise 2: Linear Discriminant Analysis (LDA)  (4 points)
Implement the calculation of the LDA classifier as a Python function `train_LDA`.  The function should take two arguments, the first being the data matrix $\bf{X}$ where each column is a data point ($\bf{x_k}$), and the second being class labels of the data points. Two output arguments should return the weight vector **`w`** and bias `b`.

In [3]:
def train_LDA(X, y):
    '''
    Synopsis:
        w, b= train_LDA(X, y)
    Arguments:
        X: data matrix (features X samples)
        y: labels with values 0 and 1 (1 x samples)
    Output:
        w: LDA weight vector
        b: bias term
    '''
    print("train_LDA(): LDA call")
    u1 = np.mean(X[:, y == 0])
    print("train_LDA(): Mean1 calc:", u1)
    u2 = np.mean(X[:, y == 1])
    print("train_LDA(): Mean2 calc:", u2 )
    
    #From np.cov documentation:  each row represents a variable, 
    #with observations in the columns
    C = np.cov(X[:, y == 1], rowvar = False)
    print("train_LDA(): Cov calc, shape:", C.shape)
    
    #w = (np.linalg.inv(C))*(u2 - u1)
    w = (np.linalg.inv(C)).dot(u2 - u1)
    print("train_LDA(): w calc, shape:", w.shape)
    b = w.T * ((u1 + u2)/2)
    print("train_LDA(): b calc, shape:", b.shape)
    print("")
    return w, b

## Exercises 3: Cross-validation with weighted loss (2 points)
Complete the implementation of `crossvalidation` by writing a loss function `loss_weighted_error` which calculates the weighted loss as explained in the lecture.

In [4]:
def crossvalidation(classifier_fcn, X, y, nFolds=10, verbose=False):
    '''
    Synopsis:
        loss_te, loss_tr= crossvalidation(classifier_fcn, X, y, nFolds=10, verbose=False)
    Arguments:
        classifier_fcn: handle to function that trains classifier as output w, b
        X:              data matrix (features X samples)
        y:              labels with values 0 and 1 (1 x samples)
        nFolds:         number of folds
        verbose:        print validation results or not
    Output:
        loss_te: value of loss function averaged across test data
        loss_tr: value of loss function averaged across training data
    '''
    nDim, nSamples = X.shape
    print("crossvalidation(): nDim:", nDim)
    print("crossvalidation(): nSamples", nSamples)
    inter = np.round(np.linspace(0, nSamples, num=nFolds + 1)).astype(int)
    perm = np.random.permutation(nSamples)
    errTr = np.zeros([nFolds, 1])
    errTe = np.zeros([nFolds, 1])

    for ff in range(nFolds):
        #Testset
        idxTe = perm[inter[ff]:inter[ff + 1] + 1]
        #Trainingset
        idxTr = np.setdiff1d(range(nSamples), idxTe)
        w, b = classifier_fcn(X[:, idxTr], y[idxTr])
        print("crossvalidation(): w shape:", w.shape)
        print("crossvalidation(): b shape:", b.shape)
        print("crossvalidation(): X shape:", X.shape)
        out = w.T.dot(X) - b
        errTe[ff] = loss_weighted_error(out[idxTe], y[idxTe])
        errTr[ff] = loss_weighted_error(out[idxTr], y[idxTr])

    if verbose:
        print('{:5.1f} +/-{:4.1f}  (training:{:5.1f} +/-{:4.1f})  [using {}]'.format(errTe.mean(), errTe.std(),
                                                                                     errTr.mean(), errTr.std(), 
                                                                                     classifier_fcn.__name__))
    return np.mean(errTe), np.mean(errTr)

In [5]:
def loss_weighted_error(out, y):
    '''
    Synopsis:
        loss= loss_weighted_error( out, y )
    Arguments:
        out:  output of the classifier
        y:    true class labels
    Output:
        loss: weighted error
    '''
#test this function with knowing what the output should be: only one class and the output should be 50%, 
#same output as y --> error should be 0
    err0 = 0
    err1 = 0

    for i in range(0, len(y)):
        if(y[i] == 0):
            if(y[i] != out[i]):
                err0+=1
        else:
            if(y[i] != out[i]):
                err1+=1
    loss = (err0 + err1)/2
    return loss

## Preparation: Load Data

In [6]:
fname = 'erp_hexVPsag.npz'
cnt, fs, clab, mnt, mrk_pos, mrk_class, mrk_className = bci.load_data(fname)

## Exercise 4: Classification of Temporal Features  (3 points)
Extract as temporal features from single channels the epochs of the time interval 0 to 1000 ms. Determine the error of classification with LDA and with NCC on those features using 10-fold cross-validation for each single channel. Display the resulting (test) error rates for all channel as scalp topographies (one for LDA and one for NCC).

In [7]:
ref_ival = [-100, 0]
ival = [0, 1000]
epo, epo_t = bci.makeepochs(cnt, fs, mrk_pos, ival)
epo = bci.baseline(epo, epo_t, ref_ival)

for i in range(0, epo.shape[1]):
    tmp = epo[:,i,:]
    loss_te, loss_tr = crossvalidation(train_LDA, tmp , mrk_class, nFolds=10, verbose=False)

crossvalidation(): nDim: 101
crossvalidation(): nSamples 1200
train_LDA(): LDA call
train_LDA(): Mean1 calc: 0.4921431565983542
train_LDA(): Mean2 calc: -0.4291856124303542
train_LDA(): Cov calc, shape: (879, 879)
train_LDA(): w calc, shape: (879, 879)
train_LDA(): b calc, shape: (879, 879)

crossvalidation(): w shape: (879, 879)
crossvalidation(): b shape: (879, 879)
crossvalidation(): X shape: (101, 1200)


ValueError: shapes (879,879) and (101,1200) not aligned: 879 (dim 1) != 101 (dim 0)

## Exercise 5: Classification of Spatial Features  (4 points)
Perform classification (*target* vs. *nontarget*) on spatial features (average across time within a 50 ms interval) in a time window that is shifted from 0 to 1000 ms in steps of 10 ms, again with both, LDA and NCC. Visualize the time courses of the classification error. Again, use 10-fold cross-validation. Here, use a baseline correction w.r.t. the prestimulus interval -100 to 0 ms.